# Acesso ao SD

> Nota: esse Notebook usa o [kernel Bash](https://pypi.org/project/bash_kernel/)

Objetivo: rodar instâncias do servidor JupyterLab (JL) no supercomputador Santos Dumont (SD), e desta forma ter acesso aos recursos do SD através do cliente JL (web-based IDE) rodando no navegador web na máquina local.

## Exemplos de configuração do ambiente

### Na máquina local

Pré-requisito: distribuição Python (Anaconda ou Intel) instalada e funcionando na máquina local

* https://www.anaconda.com/products/individual
* https://software.intel.com/content/www/us/en/develop/tools/oneapi/components/distribution-for-python.html


### Exemplo de instalação do Jupyter-Lab

Distribuição Intel (que inclui outros compiladores e ferramentas Intel)

* https://software.intel.com/content/www/us/en/develop/documentation/installation-guide-for-intel-oneapi-toolkits-linux/top/installation/install-using-package-managers/apt.html

Usando Ubuntu 20.04

```
...
sudo apt install intel-basekit intel-hpckit
sudo chmod -R 777 /opt/intel/oneapi/intelpython
source /opt/intel/oneapi/setvars.sh
conda install jupyterlab
jupyter-lab
```


### Exemplos de configuração do VPN, SSH, e JupyterLab

* https://github.com/efurlanm/hpc/blob/master/sd/access_configuring.ipynb
* https://www.hostinger.com.br/tutoriais/conexao-ssh-sem-senha
* https://github.com/efurlanm/hpc/blob/master/sd/access_using_jupyterlab.ipynb

O SSH foi configurado para chave ao invés de senha, tanto na máquina local quanto no SD para acessar o sdumont18. Além disso o arquivo `~/.ssh/config` foi configurado:

    Host sd
      HostName login.sdumont.lncc.br
      User <NOME_DO_USUARIO>
      ControlMaster auto
      ControlPath ~/.ssh/remote_sd

Configuração deste Notebook (essa célula tem que ser executada sempre):

In [46]:
bind 'set enable-bracketed-paste off'

## Acessando o Santos Dumont

* Primeiro passo: usando o Gerenciador de Rede, fazer a conexão ao VPN Cisco usando vpnc

Ao fazer login normalmente, uma máquina de login é selecionada aleatoriamente (geralmente sdumont11, sdumont13, ou sdumont14). O comando a seguir abre uma conexão mestre ("SSH Socket") em um dos nós, de tal forma que todos os comandos subsequentes sejam realizados neste mesmo nó:

In [47]:
ssh -Nf sd    # não esquecer de ligar o VPN antes

Mostra em qual nó estamos conectados:

In [48]:
ssh sd 'hostname'

sdumont13


## Primeira sessão

* no nó de login B710
* o comando `ssh` a seguir, executa os seguintes comandos no SD:
    * carrega o módulo anaconda
    * carrega uma instância do servidor JupyterLab, que fica ativo na porta 20210 (pode escolher outra)
* uma instância JupyterLab fica rodando na máquina de login onde a conexão mestre está apontando

In [49]:
ssh -T sd <<'EOF'
module load anaconda3
jupyter-lab --no-browser --port=20210 --ip=0.0.0.0 > ~/20210.log 2>&1 &
EOF

## Segunda sessão

* no nó de login Sequana (sdumont18)
* entra no nó Sequana sdumont18 usando o comando ssh que é executado na máquina de login
* carrega uma nova instância do servidor JupyterLab, que fica ativo no nó sdumont18 na porta 20211

In [50]:
ssh -T sd <<'EOF1'
ssh -Ti ~/.ssh/id_sd_ed25519 sdumont18 <<'EOF2'
module load anaconda3
jupyter-lab --no-browser --port=20211 --ip=0.0.0.0 > ~/20211.log 2>&1 &
EOF2
EOF1

## Encaminhamento de portas

Localmente o acesso é feito usando as portas 8889 e 8890 (pode escolher outras)

B710

In [51]:
ssh -NfTL 8889:localhost:20210 sd

Sequana

In [52]:
ssh -NfTL 8890:sdumont18:20211 sd

## Montagem do sistema de arquivos remoto

* É independente do Jupyter
* É importante criar os diretórios `/pr` e `/scratch` na máquina local, para evitar links quebrados

In [53]:
sshfs sd:/prj     /prj     -o workaround=rename,uid=1000,gid=1000
sshfs sd:/scratch /scratch -o workaround=rename,uid=1000,gid=1000

## Links para os clientes JupyterLab na máquina local

* se não aparecer nada ou aparecer em branco, tem que esperar um pouco para o servidor subir (pode demorar minutos)
* se assim mesmo o link não aparecer, ou der erro: mata os processos e tenta de novo

B710

In [54]:
tmp=$(ssh sd 'cat ~/20210.log | grep "     or"')
echo ${tmp/#*'20210'/'http://localhost:8889'}

http://localhost:8889/?token=33cf5520a1035df2b689aa9cc65872e19cbc99adc9805231


Sequana

In [55]:
tmp=$(ssh sd 'cat ~/20211.log | grep "     or"')
echo ${tmp/#*'20211'/'http://localhost:8890'}

http://localhost:8890/?token=d70d59513d13c6c2d74c37f1bbd84106675683b15cb9c631


# Sair

### Desmonta o sistema de arquivos remoto, e encerra o encaminhamento de portas

In [56]:
sudo umount -fl /prj /scratch

In [ ]:
pkill -f 8889:

In [ ]:
pkill -f 8890:

### Encerra a conexão

Uma forma é usar o menu do cliente JupyterLab e selecionar "File > Shut Down" para cada instância do servidor, e em seguida encerrar a conexão mestre:

In [59]:
ssh -O exit sd

Exit request sent.


Outra forma é matar os processos e em seguida encerrar a conexão mestre:

In [20]:
ssh sd 'pkill -u ${USER} -f jupyter'

In [21]:
ssh sd 'ssh -i ~/.ssh/id_sd_ed25519 sdumont18 "pkill -u ${USER} -f jupyter"'

In [12]:
ssh -O exit sd

Exit request sent.


# Verifica processos ativos nos nós

In [44]:
ssh sd 'ssh sdumont11 "ps -u ${USER}"; exit'

  PID TTY          TIME CMD
22557 ?        00:00:00 sshd
28343 ?        00:00:00 sshd
28344 ?        00:00:00 ps


In [40]:
ssh sd 'ssh sdumont13 "ps -u ${USER}"; exit'

  PID TTY          TIME CMD
 2742 ?        00:00:00 sshd
 5345 ?        00:00:00 bash
 5363 ?        00:00:00 ssh
 5367 ?        00:00:00 sshd
 5368 ?        00:00:00 ps
 6531 ?        00:00:00 sshd
 7814 ?        00:00:00 sshd
 8467 ?        00:00:00 sshd


In [41]:
ssh sd 'ssh sdumont14 "ps -u ${USER}"; exit'

  PID TTY          TIME CMD
 5013 ?        00:00:00 sshd
 5142 ?        00:00:00 ps
19269 ?        00:00:00 sshd
22876 ?        00:00:00 sshd


In [42]:
ssh sd 'ssh -i ~/.ssh/id_sd_ed25519 sdumont18 "ps -u ${USER}"; exit'

   PID TTY          TIME CMD
 29142 ?        00:00:00 sshd
 29143 ?        00:00:00 ps
